In [1]:
import os 
import numpy as np
import time
import anndata as ad
import pandas as pd 
import scanpy as sc
import parc
import timeit

os.chdir("/Users/bombina2/github/multiplex-analysis-web-apps")

df = pd.read_csv("input/measurementsthymus-exported.csv")
list(df.columns)
meta = df.iloc[:, :4]
list(meta.columns)
mat = df.iloc[:, 4:]
list(mat.columns)
meta = meta.rename(columns={"Image": "Image ID_(standardized)", 
                            "Centroid X µm": "Centroid X (µm)_(standardized)", 
                            "Centroid Y µm": "Centroid Y (µm)_(standardized)"})

adata = ad.AnnData(mat)
adata.obs = meta

/Users/bombina2/miniconda3/envs/mawa-2024-02-08/lib/python3.11/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [2]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
adata.raw = adata
adata_beforeHVG =adata.copy()

/Users/bombina2/miniconda3/envs/mawa-2024-02-08/lib/python3.11/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [3]:
sc.pp.scale(adata, max_value=5)
sc.tl.pca(adata, svd_solver='arpack', n_comps=10)

In [4]:
res = adata.obsm['X_pca']
print('size of  output', res.shape, type(res))

size of  output (1274717, 10) <class 'numpy.ndarray'>


In [12]:
%%time
p = parc.PARC(res, random_seed=42, num_threads = 8)
p.run_PARC()
adata.obs['parc'] = [str(i) for i in p.labels]

input data has shape 1274717 (samples) x 10 (features)
knn struct was not available, so making one
commencing global pruning
commencing community detection
partition type MVP
list of cluster labels and populations 22 [(0, 128895), (1, 116190), (2, 115408), (3, 114353), (4, 92568), (5, 83784), (6, 75065), (7, 73722), (8, 67564), (9, 63446), (10, 58578), (11, 52044), (12, 45175), (13, 42862), (14, 41325), (15, 28292), (16, 24299), (17, 22422), (18, 19826), (19, 5202), (20, 3468), (21, 229)]
time elapsed 306.2 seconds
CPU times: user 8min 54s, sys: 14 s, total: 9min 8s
Wall time: 5min 6s


In [11]:
%%time
parc1 = parc.PARC(adata.obsm['X_pca'], small_pop = 50, num_threads = 8, knn=10)
parc1.run_PARC()

input data has shape 1274717 (samples) x 10 (features)
knn struct was not available, so making one
commencing global pruning
commencing community detection
partition type MVP
list of cluster labels and populations 61 [(0, 43544), (1, 39766), (2, 36064), (3, 35386), (4, 35258), (5, 34466), (6, 32335), (7, 32355), (8, 30992), (9, 30895), (10, 30292), (11, 30150), (12, 30281), (13, 28994), (14, 28333), (15, 28371), (16, 28185), (17, 27896), (18, 27289), (19, 26978), (20, 25878), (21, 24675), (22, 24449), (23, 24251), (24, 24347), (25, 23940), (26, 22753), (27, 21566), (28, 20997), (29, 20321), (30, 20163), (31, 19617), (32, 19622), (33, 18642), (34, 19178), (35, 18752), (36, 17999), (37, 18087), (38, 17757), (39, 16921), (40, 16296), (41, 16345), (42, 16093), (43, 15660), (44, 14584), (45, 14663), (46, 13356), (47, 12974), (48, 12834), (49, 12923), (50, 10618), (51, 10068), (52, 9729), (53, 9284), (54, 9094), (55, 7670), (56, 7508), (57, 5573), (58, 1411), (59, 226), (60, 63)]
time elapse

In [14]:
%%time
graph = p.knngraph_full()

CPU times: user 32.9 s, sys: 1.29 s, total: 34.2 s
Wall time: 7.92 s


In [17]:
%%time
embedding= p.run_umap_hnsw(res, graph, random_state = 1)
print('completed embedding')

a,b, spread, dist 1.5769434603113077 0.8950608779109733 1.0 0.1


TypeError: simplicial_set_embedding() missing 3 required positional arguments: 'densmap', 'densmap_kwds', and 'output_dens'

In [19]:
%%time
import umap
knn_umap = umap.UMAP(n_jobs = 8).fit_transform(res)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [6]:
%%time
import umap
res_1 = adata.obsm['X_pca'].T

t_umap = umap.UMAP(n_jobs = 8).fit_transform(res_1)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
/Users/bombina2/miniconda3/envs/mawa-2024-02-08/lib/python3.11/site-packages/umap/umap_.py:2437: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


CPU times: user 2.3 s, sys: 95.1 ms, total: 2.4 s
Wall time: 2.38 s


In [7]:
t_umap

array([[ 18.644636, -10.383589],
       [ 17.833063,  -8.508753],
       [ 17.579248, -10.41281 ],
       [ 16.532759,  -9.930926],
       [ 18.239628,  -9.478325],
       [ 19.057205,  -9.379115],
       [ 16.968237,  -8.94392 ],
       [ 17.49615 ,  -9.595024],
       [ 18.522205,  -8.685949],
       [ 19.38408 , -10.420888]], dtype=float32)